Set Shots - Expected Miss Model - Model Tuning and Building - GBM

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import joblib
import warnings
warnings.filterwarnings('ignore')

from expected_score_model.config import miss_set_shots_file_path
from expected_score_model.domain.contracts.modelling_data_contract import ModellingDataContract
from expected_score_model.domain.modelling.hyperparameter_tuning import XGBHyperparameterTuner
from expected_score_model.domain.modelling.supermodel import SuperXGBClassifier
from expected_score_model.domain.modelling.optuna_xgb_param_grid import OptunaXGBParamGrid

pd.options.display.max_rows = 100
pd.options.display.max_columns = 999

%load_ext autoreload
%autoreload 2

Model Version

In [3]:
model_version = 6
model_name = 'expected_miss_set'
model_file_name = model_name + '_v' + str(model_version)

model_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/models"
prediction_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/predictions/"

preprocessor_file_name = 'preprocessor_v' + str(model_version)
preprocessor_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/preprocessors/"

In [3]:
RESPONSE = ModellingDataContract.RESPONSE_MISS

In [4]:
FEATURES = ModellingDataContract.feature_list_set_miss

In [5]:
MONOTONE_CONSTRAINTS = ModellingDataContract.monotone_constraints_set_miss

Load Data

In [6]:
df_modelling = pd.read_csv(miss_set_shots_file_path)
df_modelling.tail()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Season,Goal,Behind,Miss,Score,Event_Type1,Set_Shot,x0,y0,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,MissTrainingSet,MissTestSet,MissValidationSet
16981,186,possGain,behind,1447.0,4,185,224.0,Collingwood,Collingwood,Bobby Hill,Bobby_Hill,Kick,32.0,0.0,ineffective,True,NaN,141,160,Collingwood,Brisbane Lions,right,2023F4_Collingwood_BrisbaneLions,2023F4,NaN,2023.0,0,1,0,1,Uncontested Mark,True,32.0,0.0,48.000000,0.000000,0.133929,True,False,False
16982,196,possGain,behind,1527.0,4,425,476.0,Brisbane Lions,Brisbane Lions,Joe Daniher,Joe_Daniher,Kick,50.0,-19.0,ineffective,True,NaN,141,160,Collingwood,Brisbane Lions,right,2023F4_Collingwood_BrisbaneLions,2023F4,NaN,2023.0,0,1,0,1,Mark On Lead,True,50.0,-19.0,35.510562,0.564569,0.153507,False,False,True
16983,210,throwIn,behind,1610.0,4,804,841.0,Brisbane Lions,Brisbane Lions,Eric Hipwood,Eric_Hipwood,Kick,36.0,37.0,ineffective,True,NaN,141,160,Collingwood,Brisbane Lions,right,2023F4_Collingwood_BrisbaneLions,2023F4,NaN,2023.0,0,1,0,1,Uncontested Mark,True,36.0,37.0,57.489129,0.699193,0.085469,True,False,False
16984,224,possGain,goal,1740.0,4,1290,1348.0,Collingwood,Collingwood,Steele Sidebottom,Steele_Sidebottom,Kick,37.0,34.0,effective,True,NaN,141,160,Collingwood,Brisbane Lions,right,2023F4_Collingwood_BrisbaneLions,2023F4,NaN,2023.0,1,0,0,6,Mark On Lead,True,37.0,34.0,54.817880,0.669043,0.091894,True,False,False
16985,233,ballUp,goal,1811.0,4,1591,1629.0,Brisbane Lions,Brisbane Lions,Joe Daniher,Joe_Daniher,Kick,54.0,2.0,effective,True,NaN,141,160,Collingwood,Brisbane Lions,right,2023F4_Collingwood_BrisbaneLions,2023F4,NaN,2023.0,1,0,0,6,Uncontested Mark,True,54.0,2.0,26.076810,0.076772,0.248447,False,False,True


In [7]:
df_modelling[['ballUp', 'centreBounce', 'kickIn', 'possGain', 'throwIn']] = pd.get_dummies(df_modelling['Initial_State'])

In [8]:
training_data = df_modelling[df_modelling[RESPONSE+"TrainingSet"]]
test_data = df_modelling[df_modelling[RESPONSE+"TestSet"]]
cal_data = df_modelling[df_modelling[RESPONSE+"ValidationSet"]]

In [9]:
X_train, y_train = training_data.drop(columns=[RESPONSE]), training_data[RESPONSE]
X_test, y_test = test_data.drop(columns=[RESPONSE]), test_data[RESPONSE]
X_cal, y_cal = cal_data.drop(columns=[RESPONSE]), cal_data[RESPONSE]

In [10]:
y_train.mean(), y_test.mean(), y_cal.mean()

(0.1295308187672493, 0.12625073572689818, 0.12214863870493009)

In [11]:
X_test.head()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Season,Goal,Behind,Score,Event_Type1,Set_Shot,x0,y0,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,MissTrainingSet,MissTestSet,MissValidationSet,ballUp,centreBounce,kickIn,possGain,throwIn
1,28,possGain,behind,295.0,1,859,911.0,Sydney,Sydney,Sam Reid,Sam_Reid,Kick,52.0,35.0,ineffective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,0,1,1,Contested Mark,True,52.0,35.0,43.600459,0.931882,0.088007,False,True,False,False,False,False,True,False
17,168,centreBounce,miss,1480.0,3,1850,1899.0,Brisbane Lions,Brisbane Lions,Eric Hipwood,Eric_Hipwood,Kick,52.0,-31.0,clanger,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,0,0,0,Mark On Lead,True,52.0,-31.0,40.459857,0.872894,0.102289,False,True,False,False,True,False,False,False
19,189,kickIn,goal,1639.0,4,390,439.0,Brisbane Lions,Brisbane Lions,Lincoln McCarthy,Lincoln_McCarthy,Kick,55.0,-2.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,1,0,6,Free For,True,55.0,-2.0,23.086793,0.086738,0.281582,False,True,False,False,False,True,False,False
25,11,centreBounce,goal,103.0,1,190,249.0,Western Bulldogs,Western Bulldogs,Aaron Naughton,Aaron_Naughton,Kick,54.0,-18.0,effective,True,NaN,141,160,Collingwood,Western Bulldogs,right,202101_Collingwood_WesternBulldogs,202101,2021.0,NaN,1,0,6,Mark On Lead,True,54.0,-18.0,31.622777,0.605545,0.168122,False,True,False,False,True,False,False,False
36,139,possGain,behind,1190.0,3,205,256.0,Collingwood,Collingwood,Brody Mihocek,Brody_Mihocek,Kick,50.0,-30.0,ineffective,True,NaN,141,160,Collingwood,Western Bulldogs,right,202101_Collingwood_WesternBulldogs,202101,2021.0,NaN,0,1,1,Uncontested Mark,True,50.0,-30.0,42.426407,0.785398,0.107277,False,True,False,False,False,False,True,False


Preprocess Data

In [12]:
# preprocessor = DataPreprocessor()
# preprocessor.fit(X_train)

# X_train_preproc = preprocessor.transform(X_train)
# X_test_preproc = preprocessor.transform(X_test)

In [13]:
X_train_preproc = X_train[FEATURES]
X_test_preproc = X_test[FEATURES]
X_cal_preproc = X_cal[FEATURES]

In [14]:
X_train_preproc.shape, X_test_preproc.shape

((10870, 5), (3398, 5))

In [15]:
X_train_preproc.head()

,x0,y0,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle
0,46.0,-41.0,52.009614,0.908067,0.075999
2,45.0,16.0,36.674242,0.451453,0.158231
4,33.0,12.0,46.572524,0.260602,0.133410
5,52.0,36.0,44.407207,0.945311,0.084822
6,47.0,35.0,46.754679,0.845930,0.091187


In [16]:
X_test_preproc.head()

,x0,y0,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle
1,52.0,35.0,43.600459,0.931882,0.088007
17,52.0,-31.0,40.459857,0.872894,0.102289
19,55.0,-2.0,23.086793,0.086738,0.281582
25,54.0,-18.0,31.622777,0.605545,0.168122
36,50.0,-30.0,42.426407,0.785398,0.107277


Optuna Hyperparameter Tuning Class - HyperParameterTuner & XGBHyperparameterTuner

In [17]:
xgb_tuner = XGBHyperparameterTuner(X_train_preproc, y_train, monotonicity_constraints=MONOTONE_CONSTRAINTS)

In [18]:
xgb_tuner.training_data.head()

,x0,y0,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle
0,46.0,-41.0,52.009614,0.908067,0.075999
2,45.0,16.0,36.674242,0.451453,0.158231
4,33.0,12.0,46.572524,0.260602,0.133410
5,52.0,36.0,44.407207,0.945311,0.084822
6,47.0,35.0,46.754679,0.845930,0.091187


In [19]:
xgb_tuner.tune_hyperparameters()

[I 2023-10-21 14:13:34,460] A new study created in memory with name: no-name-16c672c7-b558-4d99-9306-c7b69f27e574
[I 2023-10-21 14:13:34,493] Trial 0 finished with value: 0.3724391292551068 and parameters: {'max_depth': 5, 'min_child_weight': 81, 'eta': 0.020555732641452978, 'gamma': 0.004404183865070571, 'lambda': 0.0016807929239833616, 'alpha': 3.056376422749878, 'subsample': 0.7522158551145055, 'colsample_bytree': 0.4215061658094299}. Best is trial 0 with value: 0.3724391292551068.
[I 2023-10-21 14:13:34,505] Trial 1 finished with value: 0.3869183821230906 and parameters: {'max_depth': 11, 'min_child_weight': 10, 'eta': 0.04685748253166966, 'gamma': 0.012366970558110413, 'lambda': 0.0011570510477636531, 'alpha': 0.5768383802859032, 'subsample': 0.6733707733389364, 'colsample_bytree': 0.31424724134451887}. Best is trial 0 with value: 0.3724391292551068.
[I 2023-10-21 14:13:34,516] Trial 2 finished with value: 0.3239112127225797 and parameters: {'max_depth': 20, 'min_child_weight': 38

Number of finished trials:  1000
Best trial:
  Value: 0.29160494744996257
  Params: 
    max_depth: 14
    min_child_weight: 33
    eta: 0.6470506272583519
    gamma: 0.2665755840612027
    lambda: 0.012690168061405447
    alpha: 3.2000156002308664
    subsample: 0.7678748296394052
    colsample_bytree: 0.5653380692076304


In [20]:
params = xgb_tuner.get_best_params()
params

{'max_depth': 14,
 'min_child_weight': 33,
 'eta': 0.6470506272583519,
 'gamma': 0.2665755840612027,
 'lambda': 0.012690168061405447,
 'alpha': 3.2000156002308664,
 'subsample': 0.7678748296394052,
 'colsample_bytree': 0.5653380692076304}

Training Model - SuperXGBClassifier class for training and predictions

In [21]:
params['objective'] = OptunaXGBParamGrid.error
params['num_rounds'] = 1000
params['early_stopping_rounds'] = 50
params['verbosity'] = 1
params['monotone_constraints'] = MONOTONE_CONSTRAINTS

In [22]:
super_xgb = SuperXGBClassifier(X_train = X_train_preproc, 
                               y_train = y_train, 
                               X_test = X_test_preproc, 
                               y_test = y_test,
                               X_cal = X_cal_preproc,
                               y_cal = y_cal,
                               params = params)

In [23]:
super_xgb.fit()

[0]	validation_0-logloss:0.34531	validation_1-logloss:0.34247
[1]	validation_0-logloss:0.33183	validation_1-logloss:0.33233
[2]	validation_0-logloss:0.32807	validation_1-logloss:0.32919
[3]	validation_0-logloss:0.31987	validation_1-logloss:0.32241
[4]	validation_0-logloss:0.31955	validation_1-logloss:0.32222
[5]	validation_0-logloss:0.31558	validation_1-logloss:0.31954
[6]	validation_0-logloss:0.31517	validation_1-logloss:0.31901
[7]	validation_0-logloss:0.31393	validation_1-logloss:0.31850
[8]	validation_0-logloss:0.31392	validation_1-logloss:0.31841
[9]	validation_0-logloss:0.31392	validation_1-logloss:0.31837
[10]	validation_0-logloss:0.31293	validation_1-logloss:0.31846
[11]	validation_0-logloss:0.31292	validation_1-logloss:0.31860
[12]	validation_0-logloss:0.31281	validation_1-logloss:0.31861
[13]	validation_0-logloss:0.31280	validation_1-logloss:0.31856
[14]	validation_0-logloss:0.31240	validation_1-logloss:0.31824
[15]	validation_0-logloss:0.31234	validation_1-logloss:0.31853
[1

In [24]:
super_xgb.xgb_model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.5653380692076304, device=None,
              early_stopping_rounds=50, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0.2665755840612027,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.64705...2583519,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=14, max_leaves=None,
              min_child_weight=33, missing=nan,
              monotone_constraints={'Angle_to_Middle_Goal': 1,
                                    'Distance_to_Middle_Goal': 1,
                                    'Visible_Goal_Angle': -1, 'x0': 1},
              multi_strategy=None, n_estimators=1000, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [25]:
super_xgb.xgb_model.get_booster().feature_names

['x0',
 'y0',
 'Distance_to_Middle_Goal',
 'Angle_to_Middle_Goal',
 'Visible_Goal_Angle']

In [26]:
train_preds = super_xgb.predict(X_train_preproc)
test_preds = super_xgb.predict(X_test_preproc)

In [27]:
train_probas = super_xgb.predict_proba(X_train_preproc)[:, 1]
test_probas = super_xgb.predict_proba(X_test_preproc)[:, 1]
cal_probas = super_xgb.predict_proba(X_cal_preproc)[:, 1]

In [28]:
super_xgb.calibrate()

In [29]:
train_cal_probas = super_xgb.predict_proba(X_train_preproc, calibrate=True)
test_cal_probas = super_xgb.predict_proba(X_test_preproc, calibrate=True)
cal_cal_probas = super_xgb.predict_proba(X_cal_preproc, calibrate=True)

Check Average Predictions

In [30]:
train_probas.mean(), training_data[RESPONSE].mean(), train_cal_probas.mean()

(0.12976122, 0.1295308187672493, 0.1280081948535256)

In [31]:
test_probas.mean(), test_data[RESPONSE].mean(), test_cal_probas.mean()

(0.13085213, 0.12625073572689818, 0.12880264417164083)

In [32]:
cal_probas.mean(), cal_data[RESPONSE].mean(), cal_cal_probas.mean()

(0.12363563, 0.12214863870493009, 0.12214858574266423)

Export model

In [33]:
super_xgb.export_model(model_output_path + "/" + model_file_name + ".joblib")

In [5]:
model_output_path + "/" + model_file_name + ".joblib"

'/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/models/expected_miss_set_v6.joblib'

In [4]:
joblib.load(model_output_path + "/" + model_file_name + ".joblib")

Export data and predictions

In [34]:
train_info = training_data[['Match_ID', "Chain_Number", "Order", RESPONSE]].reset_index(drop = True)
train_export = pd.concat([train_info, X_train_preproc.reset_index(drop=True)], axis=1)
train_export['xgb_preds'] = train_preds
train_export['xgb_probas'] = train_probas
train_export['xgb_probas_cal'] = train_cal_probas
train_export.to_csv(prediction_output_path + 'train_predictions_' + model_file_name + '.csv', index = False)
train_export.head()

,Match_ID,Chain_Number,Order,Miss,x0,y0,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,xgb_preds,xgb_probas,xgb_probas_cal
0,202101_BrisbaneLions_Sydney,17,183.0,0,46.0,-41.0,52.009614,0.908067,0.075999,0,0.338251,0.320444
1,202101_BrisbaneLions_Sydney,34,354.0,0,45.0,16.0,36.674242,0.451453,0.158231,0,0.021770,0.023778
2,202101_BrisbaneLions_Sydney,59,578.0,0,33.0,12.0,46.572524,0.260602,0.133410,0,0.078913,0.085519
3,202101_BrisbaneLions_Sydney,62,621.0,0,52.0,36.0,44.407207,0.945311,0.084822,0,0.321053,0.306856
4,202101_BrisbaneLions_Sydney,72,696.0,0,47.0,35.0,46.754679,0.845930,0.091187,0,0.243035,0.242095


In [35]:
test_info = test_data[['Match_ID', "Chain_Number", "Order", RESPONSE]].reset_index(drop = True)
test_export = pd.concat([test_info, X_test_preproc.reset_index(drop=True)], axis=1)
test_export['xgb_preds'] = test_preds
test_export['xgb_probas'] = test_probas
test_export['xgb_probas_cal'] = test_cal_probas
test_export.to_csv(prediction_output_path + 'test_predictions_' + model_file_name + '.csv', index = False)
test_export.head()

,Match_ID,Chain_Number,Order,Miss,x0,y0,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,xgb_preds,xgb_probas,xgb_probas_cal
0,202101_BrisbaneLions_Sydney,28,295.0,0,52.0,35.0,43.600459,0.931882,0.088007,0,0.315726,0.302599
1,202101_BrisbaneLions_Sydney,168,1480.0,1,52.0,-31.0,40.459857,0.872894,0.102289,0,0.089349,0.096407
2,202101_BrisbaneLions_Sydney,189,1639.0,0,55.0,-2.0,23.086793,0.086738,0.281582,0,0.004116,0.004353
3,202101_Collingwood_WesternBulldogs,11,103.0,0,54.0,-18.0,31.622777,0.605545,0.168122,0,0.024325,0.026599
4,202101_Collingwood_WesternBulldogs,139,1190.0,0,50.0,-30.0,42.426407,0.785398,0.107277,0,0.162284,0.168860
